In [1]:
!pip install surprise
!pip install scikit-surprise

In [2]:
%%time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix
from tqdm.auto import tqdm
from collections import Counter
from surprise import Dataset, Reader
import time
from surprise import KNNBasic

CPU times: total: 266 ms
Wall time: 279 ms


c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\track_artists.csv"
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\surprize_artist_KNNBasic_target"
target_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\surprize_artist_KNNBasic_pred"

In [4]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")

In [5]:
%%time
df = pd.read_csv(df_path)
df.index = df['trackId']

CPU times: total: 46.9 ms
Wall time: 50.8 ms


In [6]:
%%time

list_coord = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(10000), size=1000, replace=False)
val_id.sort()
all_artist = set()
ls_artists = []

with open(train_path) as f:
    lines = f.readlines()
    idx = 0 
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        tracks_arr = np.array(tracks)
        tracks_arr = tracks_arr.astype(np.int32)
        artist_arr = np.array([df.at[track, 'artistId'] for track in tracks_arr])
        if idx in val_id:
            target_val.append(artist_arr[-1])
            artist_arr = artist_arr[:-1]
        ls_artists.extend(artist_arr)
        all_artist = all_artist.union(set(artist_arr))
        artist_cnt = Counter(artist_arr)
        list_coord_user = [(idx, i[0], i[1]) for i in artist_cnt.items()]
        list_coord.extend(list_coord_user)
        idx += 1
        if idx > 10000:
            break

  1%|▌                                                                       | 10000/1160084 [00:09<18:38, 1027.93it/s]

CPU times: total: 10.9 s
Wall time: 10.8 s


In [7]:
%%time

data = pd.DataFrame(list_coord, columns=['uid', 'iid', 'rating_row'])
data['rating_max'] = data.rating_row / data.groupby('uid')['rating_row'].transform('max')
data['rating_part'] = data.rating_row / data.groupby('uid')['rating_row'].transform('sum')

CPU times: total: 2.73 s
Wall time: 2.73 s


# Baseline

In [8]:
all_artist_cnt = [i[0] for i in Counter(ls_artists).most_common(100)]

result = [' '.join(map(str, all_artist_cnt)) + '\n' for _ in val_id]

with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

MRR@100 = 0.0144


# RATING_MAX

In [9]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 562 ms
Wall time: 574 ms


## KNNBasic

In [10]:
def optimize(k=40, 
             min_k=1,
             user_based=True,
             name="pearson_baseline",
             verbose=True):

    start_time = time.time()
    
    sim_options = {
        "name": name,
        "user_based": user_based
    }
    
    algo = KNNBasic(k=k, min_k=min_k, sim_options=sim_options, verbose=True)
    
    algo.fit(trainset)
    
    print("--- fit: %s seconds ---" % (time.time() - start_time))
    
    pred = []
    for uid in tqdm(val_id):
        pred_user = []
        for iid in all_artist:
            score = algo.predict(uid=uid, iid=iid).est
            pred_user.append((iid, score))
        pred.append(pred_user)

    res = []
    for user in tqdm(pred):
        ls = sorted(user, key=lambda x: x[1])
        temp = [i[0] for i in ls[-100:]]
        temp.reverse()
        res.append(temp)

    result = [' '.join(map(str, i)) + '\n' for i in res]


    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    calc_score(target_path, pred_path)

In [11]:
for name in ['msd', 'pearson', 'pearson_baseline']:
    for k in [10, 40, 80]:
        for min_k in [1, 10, 50]:
            print(f'name: {name}, k: {k}, min_k: {min_k}')
            optimize(k=k, 
                     min_k=min_k,
                     user_based=True,
                     name=name,
                     verbose=True)

name: msd, k: 10, min_k: 1
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.52966046333313 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 107.28it/s]


MRR@100 = 0.0064
name: msd, k: 10, min_k: 10
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.768763780593872 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 182.55it/s]


MRR@100 = 0.0822
name: msd, k: 10, min_k: 50
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.762596130371094 seconds ---


...

MRR@100 = 0.0732
name: msd, k: 80, min_k: 50
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.593165397644043 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 253.55it/s]


MRR@100 = 0.0811
name: pearson, k: 10, min_k: 1
Computing the pearson similarity matrix...
Done computing similarity matrix.
--- fit: 14.573506116867065 seconds ---


  9%|███████▏                                                                        | 90/1000 [00:44<07:29,  2.03it/s]


KeyboardInterrupt: 

In [12]:
for name in ['msd', 'pearson', 'pearson_baseline']:
    print(f'name: {name}')
    optimize(k=40, 
             min_k=10,
             user_based=True,
             name=name,
             verbose=True)

name: msd
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 12.098661422729492 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 189.37it/s]


MRR@100 = 0.0810
name: pearson
Computing the pearson similarity matrix...
Done computing similarity matrix.
--- fit: 14.608734607696533 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 293.15it/s]


MRR@100 = 0.0585
name: pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 13.261484146118164 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 251.16it/s]


MRR@100 = 0.0720


# RATING_PART

In [13]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_part']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 797 ms
Wall time: 800 ms


In [14]:
optimize(k=40, 
         min_k=10,
         user_based=True,
         name='msd',
         verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.266999959945679 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 234.26it/s]


MRR@100 = 0.0745


In [15]:
optimize(k=40, 
         min_k=10,
         user_based=True,
         name='pearson_baseline',
         verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 11.033050060272217 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 318.75it/s]


MRR@100 = 0.0688


## Best

In [16]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 641 ms
Wall time: 649 ms


In [17]:
for k in [30, 40, 50]:
    for min_k in [5, 10, 20]:
        print(f'k: {k}, min_k: {min_k}')
        optimize(k=k, 
                 min_k=min_k,
                 user_based=True,
                 name='msd',
                 verbose=True)

k: 30, min_k: 5
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.36885929107666 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 209.21it/s]


MRR@100 = 0.0759
k: 30, min_k: 10
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.142062187194824 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 224.98it/s]


MRR@100 = 0.0837
k: 30, min_k: 20
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.4156105518341064 seconds ---


...

MRR@100 = 0.0700
k: 50, min_k: 10
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.96920919418335 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 226.54it/s]


MRR@100 = 0.0791
k: 50, min_k: 20
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.858316898345947 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 231.50it/s]


MRR@100 = 0.0835


In [18]:
for k in [40, 45, 50]:
    for min_k in [25, 30, 50]:
        print(f'k: {k}, min_k: {min_k}')
        optimize(k=k, 
                 min_k=min_k,
                 user_based=True,
                 name='msd',
                 verbose=True)

k: 40, min_k: 25
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.826464414596558 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 123.14it/s]


MRR@100 = 0.0863
k: 40, min_k: 30
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.302326440811157 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 299.22it/s]


MRR@100 = 0.0856
k: 40, min_k: 50
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.6442790031433105 seconds ---


...

MRR@100 = 0.0834
k: 50, min_k: 30
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.713289499282837 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 307.16it/s]


MRR@100 = 0.0828
k: 50, min_k: 50
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.894007444381714 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 330.65it/s]


MRR@100 = 0.0831


In [19]:
for k in [35, 40, 43]:
    for min_k in [23, 25, 27]:
        print(f'k: {k}, min_k: {min_k}')
        optimize(k=k, 
                 min_k=min_k,
                 user_based=True,
                 name='msd',
                 verbose=True)

k: 35, min_k: 23
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.078680992126465 seconds ---


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 80.36it/s]


MRR@100 = 0.0867
k: 35, min_k: 25
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.936716794967651 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 297.59it/s]


MRR@100 = 0.0873
k: 35, min_k: 27
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.961191654205322 seconds ---


...

MRR@100 = 0.0836
k: 43, min_k: 25
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.8195178508758545 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 282.13it/s]


MRR@100 = 0.0842
k: 43, min_k: 27
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 6.9332122802734375 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 302.73it/s]


MRR@100 = 0.0844


## Final best
k: 35, min_k: 25<br>
Computing the msd similarity matrix...<br>
Done computing similarity matrix.<br>
--- fit: 6.936716794967651 seconds ---<br>
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:04<00:00,  1.84it/s]<br>
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 297.59it/s]<br>
MRR@100 = 0.0873